import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
#rplc imprt data , (1,cls_num+1)<> (0,cls_num) ,( [i-1, <<>> [i ) , Ntwrk/CNN1/CNN2  , ClsPrcnt , AugPrcnt 

In [2]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
rate0=900
rateNet=480                                                                  # Real rate , Load data+ '0'
cls_num=2
for i in range (1,cls_num+1):                                                  #Cls from 0to1 >>   replace:(1,cls_num+1)
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=WormsTwoClass0(i)                # Real rate    >>   Load data+'0'

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)                                                             #   4 case 1 to 0
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):                                                    #Cls from 0to1 >> (1,cls_num) 
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=int(min(clses_lens))

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [33 44]
max(train_feature_Altitude)= 4.1961
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.21316
 -0.22455 

In [4]:
np.shape(ecg)

(77, 901)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate0                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

each class and its windows =
1 33	2 44	

<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,5), int) 
rslts=np.array([])
for X in range(1,5):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2

            f_scr=np.array([67,69,69,67,68,68])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            
            scors0=np.array([61,74])                       #Class Recalls without Augmentation
            MxMn20fscr=((max(f_scr)-f_scr[0])/20)**0.7
            Cls_amnt=1+alpha*MxMn20fscr*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*Cls_amnt[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(MxMn20fscr*np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            rprt=np.empty((0,3),float)
            print('aug_amnt=\n',aug_amnt)

            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-5  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=1            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort),axis=0)
                    ''''if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort),axis=0)'''
                    #print('class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(signal.resample(Xtrain,rateNet,axis=1))
                y_train=np.array(ytrain)
                X_test=np.array(signal.resample(Xtest,rateNet,axis=1))
                y_test=np.array(ytest)
                X_valid=np.array(signal.resample(Xtrain,rateNet,axis=1))
                y_valid=np.array(ytrain)

                if min(y_train)==1:
                    y_train=y_train-1
                    y_test=y_test-1
                    y_valid =y_valid-1
                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                #VGG16_1D

                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :

                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data

                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=np.argmax(predict_x,axis=1)           # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 45
aug_amnt=
 [[1 1 0 0 0]
 [0 0 0 0 0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 1  alpha 1  Beta 1  average= [70 69 70]  max= [74 73 73]  var= [7 7 6]
aug_amnt=
 [[1 1 0 0 0]
 [0 0 0 0 0]]
6/6 [==============================] - 0s 2ms/step
rslts= X 1  alpha 1  Beta 2  average= [70 68 69]  max= [74 72 73]  var= [19 14 16]
aug_amnt=
 [[1 1 0 0 0]
 [0 0 0 0 0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 1  alpha 1  Beta 6  average= [71 69 70]  max= [73 71 72]  var= [2 2 1]
aug_amnt=
 [[1 1 0 0 0]
 [0 0 0 0 0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 1  alpha 2  Beta 1  average= [73 71 72]  max= [75 73 75]  var= [ 6 10  9]
aug_amnt=
 [[2 2 0 0 0]
 [0 0 0 0 0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 1  alpha 2  Beta 2  average= [67 66 67]  max= [70 69 70]  var= [6 5 5]
aug_amnt=
 [[2 2 0 0 0]
 [0 0 0 0 0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 1  alpha 2  Beta 6  average= [67 66 67] 

6/6 [==============================] - 0s 3ms/step
rslts= X 4  alpha 2  Beta 1  average= [68 66 67]  max= [70 68 69]  var= [3 3 3]
aug_amnt=
 [[16 16  0  4  4]
 [12 12  0  3  3]]
6/6 [==============================] - 0s 2ms/step
rslts= X 4  alpha 2  Beta 2  average= [64 63 64]  max= [66 65 66]  var= [2 1 1]
aug_amnt=
 [[20 20  0  0  0]
 [13 13  0  0  0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 4  alpha 2  Beta 6  average= [69 68 69]  max= [70 69 70]  var= [0 0 0]
aug_amnt=
 [[16 16  0  8  8]
 [13 13  0  6  6]]
6/6 [==============================] - 0s 3ms/step
rslts= X 4  alpha 3  Beta 1  average= [66 65 66]  max= [67 66 67]  var= [1 0 1]
aug_amnt=
 [[19 19  0  4  4]
 [14 14  0  3  3]]
6/6 [==============================] - 0s 2ms/step
rslts= X 4  alpha 3  Beta 2  average= [66 64 65]  max= [66 65 66]  var= [0 0 0]
aug_amnt=
 [[24 24  0  0  0]
 [13 13  0  0  0]]
6/6 [==============================] - 0s 3ms/step
rslts= X 4  alpha 3  Beta 6  average= [67 66 67]  max=

In [7]:
rslts

array(['X 1  alpha 1  Beta 1  average= [70 69 70]  max= [74 73 73]  var= [7 7 6]',
       'X 1  alpha 1  Beta 2  average= [70 68 69]  max= [74 72 73]  var= [19 14 16]',
       'X 1  alpha 1  Beta 6  average= [71 69 70]  max= [73 71 72]  var= [2 2 1]',
       'X 1  alpha 2  Beta 1  average= [73 71 72]  max= [75 73 75]  var= [ 6 10  9]',
       'X 1  alpha 2  Beta 2  average= [67 66 67]  max= [70 69 70]  var= [6 5 5]',
       'X 1  alpha 2  Beta 6  average= [67 66 67]  max= [69 68 69]  var= [2 3 2]',
       'X 1  alpha 3  Beta 1  average= [70 68 69]  max= [72 71 72]  var= [5 5 4]',
       'X 1  alpha 3  Beta 2  average= [69 68 69]  max= [70 69 70]  var= [1 1 1]',
       'X 1  alpha 3  Beta 6  average= [70 69 70]  max= [72 70 71]  var= [5 2 3]',
       'X 2  alpha 1  Beta 1  average= [68 66 67]  max= [71 68 70]  var= [5 4 4]',
       'X 2  alpha 1  Beta 2  average= [67 65 66]  max= [67 66 67]  var= [0 1 0]',
       'X 2  alpha 1  Beta 6  average= [66 64 66]  max= [68 67 68]  var= [2 3 3]'